In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("spark://spark-master:7077") \
    .appName("events") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0") \
    .getOrCreate()

In [14]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka1:19092,kafka2:19093,kafka3:19094") \
  .option("subscribe", "events") \
  .load()

In [4]:
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [15]:
events_df = df.selectExpr("CAST(value AS STRING)")

In [16]:
events_df.writeStream \
      .format("console") \
      .outputMode("append") \
      .start() #\
#      .awaitTermination() 